출처: https://dacon.io/codeshare/3725?dtype=recent

In [ ]:
import os
import glob
import json
from glob import glob
from tqdm import tqdm
import time
import datetime
import math


import cv2
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pycocotools.coco import COCO

import torch
import torch.optim as optim
from torch  import nn, Tensor

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from collections import defaultdict, deque

import base64
import torch.distributed as dist
from torch.utils.data import Dataset

In [ ]:
!ls

drive  sample_data


## 버전 확인

In [ ]:
print('torch :', torch.__version__)
print('torchvision : ', torchvision.__version__)
print('cv2 :', cv2.__version__)

torch : 1.10.0+cu111
torchvision :  0.11.1+cu111
cv2 : 4.1.2


## 전체 데이터 json 만들기

In [ ]:
# # # 이건 저의 개인적인 코랩이기 때문에 다르신 분들은 다시 변경하시기 바랍니다.
# # ## train_PATH와 test_PATH만 변경하면 됩니다.
# # # json 파일로 만들어서 분석이 더 용이하다고 생각되어 만든 것이니 더 좋은 의견 있으면 말씀해주시기 바랍니다.

# ## Train
# train_PATH = '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data/train/annotation'
# file_list = os.listdir(train_PATH)

# train_files = []
# for file in tqdm(file_list):
#     dir = train_PATH + '/' + file
#     json_list = glob(dir + '/' +'*.json')
#     train_files.append(json_list)

# train_json_list = []
# for files in tqdm(train_files):
#     for json_file in tqdm(files):
#         train_json_list.append(json_file)

# ## Test
# test_PATH = '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Data/test/annotations'
# file_list = os.listdir(test_PATH)

# test_files = []
# for file in tqdm(file_list):
#     dir = test_PATH + '/' + file
#     json_list = glob(dir + '/' + '*.json')
#     test_files.append(json_list)

# test_json_list = []
# for files in tqdm(test_files):
#     for json_file in tqdm(files):
#         test_json_list.append(json_file)

## Sample data에서 json 만들기

In [ ]:
## Train
train_PATH = '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Sample_data/train/annotation'
file_list = os.listdir(train_PATH)

train_files = []
for file in tqdm(file_list):
    dir = train_PATH + '/' + file
    json_list = glob(dir + '/' +'*.json')
    train_files.append(json_list)

train_json_list = []
for files in tqdm(train_files):
    for json_file in tqdm(files):
        train_json_list.append(json_file)


test_PATH = '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Sample_data/test/annotations'
file_list = os.listdir(test_PATH)
test_files = []
for file in tqdm(file_list):
    dir = test_PATH + '/' + file
    json_list = glob(dir + '/' +'*.json')
    test_files.append(json_list)

test_json_list = []
for files in tqdm(test_files):
    for json_file in tqdm(files):
        test_json_list.append(json_file)

100%|██████████| 56/56 [00:00<00:00, 358596.98it/s]

100%|██████████| 56/56 [00:00<00:00, 524288.00it/s]

100%|██████████| 56/56 [00:00<00:00, 533820.51it/s]

100%|██████████| 14/14 [00:00<00:00, 145707.83it/s]

100%|██████████| 14/14 [00:00<00:00, 48569.28it/s]

100%|██████████| 14/14 [00:00<00:00, 153316.60it/s]

100%|██████████| 4/4 [00:00<00:00, 173.28it/s]


In [ ]:
import os
import numpy as np
import torch
from PIL import Image

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms = None, mode = 'train', json_file = None):
        self.mode = mode
        self.root = root
        self.transforms = transforms
        self.json_file = json_file

    def __getitem__(self, idx):
        if self.mode == 'train':

            annot = json_file[idx]
            # Train의 경로로 이동한다.
            # 나중에 진짜 데이터를 다룰 때 사용한다.
            # PATH = root + '/' + 'train'

            with open(annot, 'r') as f:
                annot = json.loads(f.read())
                
            image_name = annot['images'][0]['file_name']
            dir = image_name.split('_')[0]

            image_PATH = self.root + '/' + 'train' + '/' +'image' +'/' + dir + '/' + image_name
            
            image = Image.open(image_PATH).convert('RGB')

            boxes = []
            segmentations = []
            labels = []

            for i in range(len(annot['annotations'])):

                segmentation = annot['annotations'][i]['segmentation'][0]
                bbox = annot['annotations'][i]['bbox']
                label = torch.tensor(annot['annotations'][i]['category_id'], dtype = np.uint8)
                xmin, ymin, width, height = bbox[0],bbox[1],bbox[2],bbox[3]
                xmin, ymin, xmax, ymax = xmin, ymin, xmin + width, ymax + height
                
                boxes.append(torch.as_tensor([xmin, ymin, xmax, ymax], dtype = torch.float32))
                segmentations.append([segmentation])
                labels.append(label)

            target = {}
            target['boxes'] = boxes
            target['labels'] = labels
            target['segmentation'] = segmentations
            target['image_id'] = image_name

            if self.transforms is not None:
                img, target = self.transforms(img, target)

            return img, target

        if self.mode == 'test':
            # Test의 경로로 이동한다.
            # PATH = root + '/' + 'test'
            # 나중에 진짜 데이터를 다룰 때 사용한다.
            annot = json_file[idx]

            ##### 이 부분은 추후에 작성하기로 하자.
            with open(annot, 'r') as f:
                annot = json.loads(f.read())

            image_name = annot['images'][0]['file_name']
            dir = image_name[:2]

            image_PATH = PATH + '/' + dir + '/' + image_name
            image = Image.open(image_PATH).convert('RGB')
            target = {}

            target['image_id'] = image_name

            return image, target

    def __len__(self):
        return len(self.json_file)

In [ ]:
root = '/content/drive/MyDrive/NIA 폐플라스틱 객체 검출 예측/Sample_data'
train_dataset = CustomDataset(root = root,json_file = train_json_list, mode = 'train')
test_dataset = CustomDataset(root = root, json_file = test_json_list, mode = 'test')

## Pytorch Tutorial을 가지고 분석을 진행해 보기

PET를 먼저 진행해 보고 추후에 나머지도 진행해 볼 예정입니다.

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        self.categories = {1 : 'pet', 2 : 'ps', 3 : 'pp', 4 : 'pe'}

        self.imgs = list(sorted(os.listdir(os.path.join(root, 'image', 'PET'))))
        self.annot = list(sorted(os.listdir(os.path.join(root, 'annotation', 'PET'))))

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, 'image','PET', self.imgs[idx])

        annot_path = os.path.join(self.root, 'annotation','PET',self.annot[idx])

        with open(annot_path, 'r') as f:
            annot = json.loads(f.read())

        img = Image.open(img_path).convert('RGB')

        # width, height로 0인 이미지 생성하여 segmentation 넣어주기
        width = annot['images'][0]['width']
        height = annot['images'][0]['height']

        #각 마스크를 target으로 사용해야함
        annot = annot['annotations']

        masked_image = np.zeros((width, height), dtpye = np.uint8)

        mask = []
        boxes = []
        labels = []
        for i in range(len(annot)):
            segmentation = annot[i]['segmentation'][0]
            xmin, ymin, width, height = annot[i]['bbox'][0], annot[i]['bbox'][1], annot[i]['bbox'][2], annot[i]['bbox'][3]
            xmax = xmin + width
            ymax = ymin + height

            label = annot[i]['category_id']

            mask.append(segmentation)
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(label)




SyntaxError: ignored